In [1]:
## Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os

output_data = "../../data/output"
input_data = "../../data/input"

## Feature: Energy Consumption

In [2]:
## Source: https://www.eia.gov/totalenergy/data/browser/?tbl=T02.02#/?f=M
filename = "MER_T02_02.csv"
filepath = os.path.join(input_data,filename)
eiadf = pd.read_csv(filepath)
print("Original count: ",len(eiadf))
# Filter rows not to be used
energy = eiadf.loc[(eiadf['Description'] == "Total Energy Consumed by the Residential Sector")
                   & 
                   (eiadf['Value'] != "Not Available")]
print("energy consumption count is : ",len(energy))
# Remove the data for month 13 (total year consumption)
e = energy[['Value', 'YYYYMM']]            # Warning msg!!
e["dates"] = e['YYYYMM'].astype(str)       # make a copy to a string type
e = e.loc[~e['dates'].str.contains(r'13$')]  # remove rows that contain month 13 in the string
e = e.drop(columns=["dates"])              # remove column added
print("After cleanup count is : ",len(e))
# dataname = "monthdata.csv"
# filesave = os.path.join(output_data,dataname)
# e.to_csv(filesave, index=False)
e= e.rename(columns={'Value':"Energy"})
e.head()

Original count:  7344
energy consumption count is :  612
After cleanup count is :  543


/anaconda3/envs/PythonData_3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Energy,YYYYMM
6756,1932.187,197301
6757,1687.255,197302
6758,1497.067,197303
6759,1177.661,197304
6760,1015.008,197305


## Feature: Solar consumption

In [3]:
## Source: https://www.eia.gov/totalenergy/data/browser/?tbl=T10.05#/?f=M
#filename = "MER_T10_05.csv"
filename = "MER_T02_02.csv"
filepath = os.path.join(input_data,filename)
eiadf = pd.read_csv(filepath)
print("Original count: ",len(eiadf))
# Filter rows not to be used
# values = eiadf.loc[(eiadf['Description'] == "Distributed Solar Energy Consumption for Electricity:  Residential Sector") & 
#                    (eiadf['Value'] != "Not Available")]
solar = eiadf.loc[(eiadf['Description'] == "Solar Energy Consumed by the Residential Sector")
                   & 
                   (eiadf['Value'] != "Not Available")]
print("Solar consumption count is : ",len(solar))
# Remove the data for month 13 (total year consumption)
s = solar[['YYYYMM','Value']]            # Warning msg!!
s["dates"] = s['YYYYMM'].astype(str)       # make a copy to a string type
s = s.loc[~s['dates'].str.contains(r'13$')]  # remove rows that contain month 13 in the string
s = s.drop(columns=["dates"])              # remove column added
print("After cleanup count is : ",len(s))
dataname = "monthdata.csv"
filesave = os.path.join(output_data,dataname)
s.to_csv(filesave, index=False)
s= s.rename(columns={'Value':"Solar"})
s.head()

Original count:  7344
Solar consumption count is :  380
After cleanup count is :  351


/anaconda3/envs/PythonData_3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,YYYYMM,Solar
3292,198901,2.798
3293,198902,3.016
3294,198903,4.156
3295,198904,4.581
3296,198905,5.057


## Feature: Temperature

In [5]:
filename = "TemperatureIndex(REDTI).csv"
filepath = os.path.join(input_data,filename)

t = pd.read_csv(filepath,index_col=False)
print("Original count: ", len(t))
#t = temps.loc[temps['YEAR']>=1984] no need to filter
print("Temperature counts: ", len(t))
t.head()

Original count:  1482
Temperature counts:  1482


,YEAR,REDTI,DEGREE DAYS,MONTH,old,YYYYMM
0,1895,66.8,1023.4,1,18951,189501
1,1896,45.5,919.8,1,18961,189601
2,1897,59.9,989.9,1,18971,189701
3,1898,40.3,894.8,1,18981,189801
4,1899,50.5,944.3,1,18991,189901


#### Extra temperature data?

In [6]:
difference_merge = pd.merge(e,t,how="outer", indicator="true")
len(difference_merge)

1482

In [7]:
difference_merge.loc[difference_merge["true"] != "both"].count()

Energy           0
YYYYMM         939
YEAR           939
REDTI          939
DEGREE DAYS    939
MONTH          939
old            939
true           939
dtype: int64

not filtering temperature .. let the merge take care of it

## Feature: Price

In [8]:
## Source: http://www.ecdms.energy.ca.gov
filename = "MER_T09_08.csv"
filepath = os.path.join(input_data,filename)

prices = pd.read_csv(filepath,index_col=False)
print("Original count: ",len(prices))
residential = prices.loc[(prices['Description']== "Average Retail Price of Electricity, Residential") & 
                         (prices['Value'] != "Not Available")]
r = residential.drop(columns=['MSN','Column_Order','Description','Unit'])
print(" residential price count: ", len(r))
r["dates"] = r['YYYYMM'].astype(str)
#remove rows tha contain 13 in the string
r = r.loc[~r['dates'].str.contains(r'13$')]
r = r.drop(columns=["dates"])
print(" after cleanup count: ", len(r))
r.head()

Original count:  3390
 residential price count:  487
 after cleanup count:  429


,YYYYMM,Value
22,197607,3.9
23,197608,3.7
24,197609,3.8
25,197610,3.9
26,197611,3.8


## Adding features = X

In [9]:
# Merge energy and solar 
merge1 = e.merge(s, on="YYYYMM", how="inner")
merge1 = merge1.rename(columns={"Value_x": "Energy", "Value_y": "Solar"})
merge1.head()

,Energy,YYYYMM,Solar
0,1972.897,198901,2.798
1,1847.709,198902,3.016
2,1771.283,198903,4.156
3,1339.662,198904,4.581
4,1174.657,198905,5.057


In [10]:
# Merge energy_solar and price 
merge2 = merge1.merge(r, on="YYYYMM", how="inner")
merge2 = merge2.rename(columns={"Value": "Price"})
merge2.head()

,Energy,YYYYMM,Solar,Price
0,2018.869,199001,2.965,7.17
1,1640.31,199002,3.196,7.48
2,1559.289,199003,4.403,7.57
3,1299.971,199004,4.853,7.69
4,1143.417,199005,5.357,7.96


In [11]:
# Merge Temperature


In [12]:
merge3 =merge2.merge(t,on="YYYYMM", how="inner")

In [13]:
merge3 = merge3[['YYYYMM','Solar', 'Energy', 'Price','REDTI']]

In [14]:
merge3.head()

,YYYYMM,Solar,Energy,Price,REDTI
0,199001,2.965,2018.869,7.17,8.8
1,199002,3.196,1640.31,7.48,22.7
2,199003,4.403,1559.289,7.57,29.8
3,199004,4.853,1299.971,7.69,26.8
4,199005,5.357,1143.417,7.96,53.9


In [15]:
print(len(merge3))
merge3.dtypes

339


YYYYMM      int64
Solar      object
Energy     object
Price      object
REDTI     float64
dtype: object

#### Process dates: months into features

In [16]:
merge3["dates"] = pd.to_datetime(merge3["YYYYMM"], format="%Y%m")

In [17]:
merge3.head()

,YYYYMM,Solar,Energy,Price,REDTI,dates
0,199001,2.965,2018.869,7.17,8.8,1990-01-01
1,199002,3.196,1640.31,7.48,22.7,1990-02-01
2,199003,4.403,1559.289,7.57,29.8,1990-03-01
3,199004,4.853,1299.971,7.69,26.8,1990-04-01
4,199005,5.357,1143.417,7.96,53.9,1990-05-01


In [18]:
merge3['year'] = merge3['dates'].dt.year
merge3['month'] = merge3['dates'].dt.month
merge3.head()

,YYYYMM,Solar,Energy,Price,REDTI,dates,year,month
0,199001,2.965,2018.869,7.17,8.8,1990-01-01,1990,1
1,199002,3.196,1640.31,7.48,22.7,1990-02-01,1990,2
2,199003,4.403,1559.289,7.57,29.8,1990-03-01,1990,3
3,199004,4.853,1299.971,7.69,26.8,1990-04-01,1990,4
4,199005,5.357,1143.417,7.96,53.9,1990-05-01,1990,5


In [19]:
data = merge3[['Solar','Energy', 'year', 'month', 'Price','REDTI']]

In [20]:
data.head()

,Solar,Energy,year,month,Price,REDTI
0,2.965,2018.869,1990,1,7.17,8.8
1,3.196,1640.31,1990,2,7.48,22.7
2,4.403,1559.289,1990,3,7.57,29.8
3,4.853,1299.971,1990,4,7.69,26.8
4,5.357,1143.417,1990,5,7.96,53.9


In [21]:
datacoded = data.copy()

X = pd.get_dummies(datacoded, columns=["month"])
X.head()

,Solar,Energy,year,Price,REDTI,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2.965,2018.869,1990,7.17,8.8,1,0,0,0,0,0,0,0,0,0,0,0
1,3.196,1640.31,1990,7.48,22.7,0,1,0,0,0,0,0,0,0,0,0,0
2,4.403,1559.289,1990,7.57,29.8,0,0,1,0,0,0,0,0,0,0,0,0
3,4.853,1299.971,1990,7.69,26.8,0,0,0,1,0,0,0,0,0,0,0,0
4,5.357,1143.417,1990,7.96,53.9,0,0,0,0,1,0,0,0,0,0,0,0


In [22]:
len(X)

339

In [23]:
dataname = "features.csv"
filesave = os.path.join(output_data,dataname)
X.to_csv(filesave, index=False)
